In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error,roc_curve,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import tensorflow.keras as keras

In [ ]:
import pickle
from tensorflow.keras.models import model_from_json
from os import path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/diabetes.csv .
diabetes = pd.read_csv("diabetes.csv")

In [ ]:
def loadModels():
  if(path.exists("drive/MyDrive/machileLearning.sav")):
        ml = pickle.load(open("drive/MyDrive/machileLearning.sav", 'rb'))
  else:
        ml = ""
  if(path.exists("models/deeplearning.json")):
        json_file = open('models/deeplearning.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        dp = model_from_json(loaded_model_json)
  else:
        dp = ""
  return ml,dp
rf,kara = loadModels()

In [ ]:
from sklearn.impute import SimpleImputer

# LECTURE DES DONNES

In [ ]:
diabetes.iloc[23]

In [ ]:
diabetes.head(5)


 À PROPOS DU JEUX DE DONNEES

Cet ensemble de données provient à l'origine d’un échantillon publié par un Institut du diabète.

L'objectif de l'ensemble de données est de prédire avec un diagnostique si un patient est atteint du diabète.

Les ensembles de données comprennent plusieurs variables prédictives médicales et une variable cible, « Outcome ». Les variables prédictives comprennent le nombre de grossesses que le patient a eues, son IMC, son taux d'insuline, son âge, etc.

Chaque ligne représente un patient et les colonnes sont :

    Grossesses: nombre de fois enceintes
    Glucose: Concentration en glucose plasmatique 2 heures dans un test de tolérance au glucose par voie orale
    BloodPressure: pression artérielle diastolique (mm Hg)
    SkinThickness: Épaisseur du pli cutané des triceps (mm)
    Insuline: insuline sérique de 2 heures (mu U / ml)
    IMC: indice de masse corporelle (poids en kg / (taille en m) ^ 2)
    DiabetesPedigreeFunction: Fonction pedigree du diabète
    Age: Age (ans)
    Résultat: Variable de classe (0 ou 1).



In [ ]:
diabetes.columns

In [ ]:
col = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

Afficher le type de donnees des colonnes

In [ ]:
diabetes.dtypes

Combien de diagnostiques on a ?




In [ ]:
diabetes['Outcome'].value_counts()

In [ ]:
for c in diabetes.columns:
    plt.figure()
    plt.hist(diabetes[c],bins=15)
    plt.xlabel(c)
    plt.ylabel('frequency')
    plt.show()



**Visualisation des donnees**

In [ ]:
target_0 = diabetes[diabetes.Outcome == 0]
plt.scatter(target_0['Glucose'],target_0['BloodPressure'],color='blue',label='target_0')


target_1 =diabetes[diabetes.Outcome == 1]
plt.scatter(target_1['Glucose'],target_1['BloodPressure'],color='red',label='target_1')


plt.legend()
plt.xlabel('Glucose')
plt.ylabel('BloodPressure')

plt.show()


on peut constater que plus le taux de glucose est elevé, plus l'enregistrement est associé à un diabète (résultat = 1, points rouges), tandis que plus il est bas, et plus il est associé à l'absence du diabète (résultat = 0, points bleus).


---

Il y a un ensemble de points de valeur 0 pour Glucose et un autre de valeur 0 pour BloodPressure. Cela n'a pas de sens physiquement. Il semble que ces données ont été remplies avec 0 alors que la valeur aurait dû être NULL.

In [ ]:
for c in diabetes.columns:
    print('Pour la  colonne ',c,'il y\'a ',diabetes[diabetes[c]==0][c].count(),'zero valeurs.')

In [ ]:
# example: plot histograms of Age for Outcome=1 and Outcome=0.
plt.figure()
plt.hist(diabetes[diabetes.Outcome==1]['Age'],bins=15,label='Diabetes',color='b',alpha=0.2)
plt.hist(diabetes[diabetes.Outcome==0]['Age'],bins=15,label='No Diabetes',color='g',alpha=0.2)
plt.xlabel('Age')
plt.ylabel('frequency')
plt.legend()
plt.show()

**Divisons de l'ensemble de donnees en dataTest et dataTrain**

  **Mais tout d'abord il faut faire un petit nettoyage des donnes**

Pour certaines de ces colonnes, zéro a un sens, comme pour le nombre de grossesses (Pregnancies) et le résultat (Outcome). Mais pour d'autres colonnes, comme BloodPressure ou BMI, zéro n'a absolument aucun sens

* Pour nos colonnes quantitaives

1.  Un traitement des données manquantes en remplaçant par la médiane
2.   Une standardisation des données (transformation de manière à ce que la moyenne soit 0 et la variance 1)



In [ ]:
np.random.seed(7)
X = diabetes.drop("Outcome",axis=1)
Y = diabetes["Outcome"]


In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
mpip = Pipeline(steps=[
    ('imputation', SimpleImputer(missing_values=0, strategy="mean")),
    ('standard',StandardScaler())                 
]
)
col_quant = col
preparation=ColumnTransformer(
    transformers=[
                  ('transf',mpip,col)
    ]
)
diab_trans = preparation.fit_transform(X)

In [ ]:

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
print("Dimension de notre dataTrain",x_train.shape)
print("Dimension de notre dataTest",x_test.shape)

# LOGISTIC REGRESSION

In [ ]:
mpi=make_pipeline(StandardScaler(),
                  SimpleImputer(missing_values=0,strategy='mean'),
                  LogisticRegression()
                  )
param_grid = {
    'simpleimputer__strategy': ['mean', 'median'],
    'logisticregression__C': [0.1, 1.0, 10, 100],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear','saga']
     }

In [ ]:
grid =GridSearchCV(mpi,param_grid=param_grid,cv=5)
resultgslr=grid.fit(x_train,y_train)

In [ ]:
print('Best Score: %s' % resultgslr.best_score_)
print('Best Hyperparameters: %s' % resultgslr.best_params_)
params1 = resultgslr.best_params_

In [ ]:
lr=LogisticRegression(C=params1['C'],solver=params1['solver'],penalty=params1['penalty'],max_iter=5000)
lr.fit(x_train,y_train)

In [ ]:
y_pred_logistic=lr.predict(x_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_logistic)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

In [ ]:
#taux de succès
acc = metrics.accuracy_score(y_test,y_pred_logistic)
err=1.0-acc
#sensibilité (ou rappel)
se = metrics.recall_score(y_test,y_pred_logistic)
print("Le rappel est : {:.3f}".format(se))
print("L erreur est : {:.3f}".format(err))
print("L accuracy est : {:.3f}".format(acc))
print('Training set Accuracy :{:.3f}'.format(lr.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(lr.score(x_test,y_test)))

In [ ]:
print('predictions:{}'.format(list(y_pred_logistic[0:20])))
print('true       :{}'.format(list(y_test[0:20].astype(int))))

In [ ]:
scores_logistic = cross_val_score(lr, x_train, y_train, cv=5,scoring='accuracy')
print("Cross-validation scores: {}".format(scores_logistic))
print("Moyenne des Taux de Success:{}".format(scores_logistic.mean()))

In [ ]:
#calcul des probas d'affectation sur ech test
probas_logistic =lr.predict_proba(x_test)
probas_logistic[0:20,:]

In [ ]:
#score de 'presence'
score_logistic=probas_logistic[:,1]
#transf en 0/1 de Y_test
pos=pd.get_dummies(y_test).values
pos[0:20,:]

# **SVR**

De nombreux algorithmes de
machine learning reposent sur des paramètres
qui ne sont pas toujours évidents à déterminer
pour obtenir les meilleurs performances sur un
jeu de données à traiter. Ex. SVM.

In [ ]:
parametres = [{'C':[0.1,1,10],'kernel':['rbf','linear']}]
grid = GridSearchCV(estimator=svm.SVC(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)

Grille de recherche. On
indique les paramètres à faire varier,
scikit-learn les croise et mesure les
performances en validation croisée

In [ ]:
C = grid.best_params_['C']
kernel = grid.best_params_['kernel']

In [ ]:
classifier = svm.SVC(C=C,kernel=kernel)
classifier.fit(x_train,y_train)

In [ ]:
y_pred_svr = classifier.predict(x_test)
#taux de succès
acc_svr = metrics.accuracy_score(y_test,y_pred_svr)
err_svr=1.0-acc_svr
#sensibilité (ou rappel)
se_svr = metrics.recall_score(y_test,y_pred_svr)
print("Le rappel est : {:.3f}".format(se_svr))
print("L erreur est : {:.3f}".format(err_svr))
print("L accuracy est : {:.3f}".format(acc_svr))
print('Training set Accuracy :{:.3f}'.format(classifier.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(classifier.score(x_test,y_test)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_svr)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# KNN

In [ ]:
parametres = {'n_neighbors':np.arange(2,25,1)}
grid = GridSearchCV(estimator=KNeighborsClassifier(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)

In [ ]:
n_neighbors = grid.best_params_['n_neighbors']

cette courbe nous montre la variance de l'erreur par rapport au nombre de neighbors pris.
Comme on le constate le nombre optimal a prendre est 11

In [ ]:
modelKNN= KNeighborsClassifier(n_neighbors=n_neighbors, weights='uniform')

maintenant on va entrainer notre model

In [ ]:
modelKNN.fit(x_train,y_train)

In [ ]:
y_pred_knn = modelKNN.predict(x_test)
#taux de succès
acc_knn = metrics.accuracy_score(y_test,y_pred_knn)
err_knn=1.0-acc_knn
#sensibilité (ou rappel)
se_knn = metrics.recall_score(y_test,y_pred_knn)
print("Le rappel est : {:.3f}".format(se_knn))
print("L erreur est : {:.3f}".format(err_knn))
print("L accuracy est : {:.3f}".format(acc_knn))
print('Training set Accuracy :{:.3f}'.format(modelKNN.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(modelKNN.score(x_test,y_test)))

In [ ]:
scores_knn = cross_val_score(modelKNN, x_train, y_train, cv=5)
print("Cross-validation scores: {}".format(scores_knn))
print("Moyenne des Taux de Success:{}".format(scores_knn.mean()))

In [ ]:
y_pred_knn=modelKNN.predict(x_test)
cm = metrics.confusion_matrix(y_test,y_pred_knn)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# **RandomForest**

In [ ]:
'''parametres = {'max_depth':np.arange(1,10,1),'n_estimators':np.arange(4,200,1)}
grid = GridSearchCV(estimator=RandomForestClassifier(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)'''

In [ ]:
rf = RandomForestClassifier(max_depth=5,n_estimators=182, random_state=0)
rf.fit(x_train, y_train)

Adjustons les max_features param , pour voir si le resultat s'ameliore


In [ ]:
y_pred_random = rf.predict(x_test)
#taux de succès
acc_random = metrics.accuracy_score(y_test,y_pred_random)
err_random=1.0-acc_random
#sensibilité (ou rappel)
se_random = metrics.recall_score(y_test,y_pred_random)
print("Le rappel est : {:.3f}".format(se_random))
print("L erreur est : {:.3f}".format(err_random))
print("L accuracy est : {:.3f}".format(acc_random))
print('Training set Accuracy :{:.3f}'.format(rf.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(rf.score(x_test,y_test)))

In [ ]:
y_pred_random=rf.predict(x_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_random)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# ROC CURVE

In [ ]:
fpr, tpr, thr = metrics.roc_curve(y_test, lr.decision_function(x_test))
plt.plot(fpr,tpr, label='logistic regression')
fpr2, tpr2, thr2 = metrics.roc_curve(y_test, classifier.decision_function(x_test))
plt.plot(fpr2,tpr2,label='svm classifier')
fpr1, tpr1, thr1 = metrics.roc_curve(y_test, modelKNN.predict_proba(x_test)[:,1])
plt.plot(fpr1,tpr1,label='knn classifier')
fpr3, tpr3, thr3 = metrics.roc_curve(y_test, rf.predict_proba(x_test)[:,1])
plt.plot(fpr1,tpr1,label='Rndome Forest classifier')
plt.title('roc curve')
plt.legend(loc = 'lower right', fontsize = 16)

# DEEP LEARNING 

In [ ]:
np.random.seed(666)
X1 = x_train
Y1 = y_train
#x_train = X

In [ ]:
y_test1 = pd.get_dummies(y_test)
y_train1 = pd.get_dummies(y_train)#Y)

In [ ]:
y_test

### MODEL DEEPLEARNING1

In [ ]:
# Our first Keras Model
model1 = Sequential()
model1.add(Dense(8,input_dim=8,activation='relu'))
model1.add(Dense(16,activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
res = model1.fit(x_train, y_train,
                     epochs=200, batch_size=16
                     ,validation_data=(x_test,y_test))

In [ ]:
score =model1.evaluate(x_test,y_test)


In [ ]:
plt.figure(figsize=[10,10])
plt.plot(res.history['loss'],'b',linewidth=3.0)
plt.plot(res.history['val_loss'],'r',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

### MODEL DEEPLEARNING2

In [ ]:
stopping_val_loss =EarlyStopping(monitor="val_loss",patience=100,mode="min")
stopping_val_1 =EarlyStopping(monitor="val_accuracy",patience=100,mode="max")
seed = 7
np.random.seed(seed)
kara = Sequential()
kara.add(Dense(50,input_dim=8, activation='relu'))
kara.add(Dropout(0.2))
kara.add(Dense(2, activation='softmax'))
kara.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = kara.fit(x_train, y_train1, epochs=150, batch_size=50,validation_data=(x_test,y_test1),callbacks=[stopping_val_loss,stopping_val_1]) # 150 epoch, 10 batch size, verbose = 2

In [ ]:
score =kara.evaluate(x_test,y_test1)


In [ ]:
# Loss Curves
plt.figure(figsize=[10,10])
plt.plot(history.history['loss'],'b',linewidth=3.0)
plt.plot(history.history['val_loss'],'r',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
# Fit with cross validation
#estimator = KerasClassifier(build_fn=kara, epochs=150, batch_size=10,verbose=0)
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(kara, X, Y, scoring='recall',cv=kfold)

In [ ]:
#print(results)

In [ ]:
predictions_deep=kara.predict(x_test)
y_pred_deep = []
for i in predictions_deep:
  y_pred_deep.append(np.argmax(i))

In [ ]:
y_test

In [ ]:

cm = confusion_matrix(y_true=y_test, y_pred=y_pred_deep)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

In [ ]:

score = kara.evaluate(x_test,y_test1)

In [ ]:
plt.scatter(x_test[:,1],y_test,color='blue',label='x_test true')
plt.scatter(x_test[:,1],y_pred_deep,color='red',label='x_test predit')


plt.legend()
plt.xlabel('Glucose')
plt.ylabel('Diabete')

plt.show()


In [ ]:
print("Score deep learning : {} and error  {}".format(score[0],score[1]))

In [ ]:
# accuracy and loss of our training set
loss, accuracy = kara.evaluate(x_train,y_train1, verbose=0)
print("Train: accuracy=%f, loss=%f" % (accuracy, loss))

In [ ]:
loss,accuracy=kara.evaluate(x_test,y_test1,verbose=0)
print("Validation:accuracy=%f,loss=%f"%(accuracy,loss))



The training accuracy came out better (82% !) but the validation accuracy is a bit far behind, which means we are in an overfitting scenario. Our model is memorizing the training samples and is no good for other samples.

Most likely we will not be able to do better than this without more training data. Luckily, real medical databases usually contain larger sets of data that suffice for building efficient neural-networks. This is one of main lessons one learns each time: a neural network needs a lot of training samples. This is why it is important for institutions to collect as much data as possible.


### model3

In [ ]:
sq = Sequential()
sq.add(Dense(12, input_dim=8, activation='relu'))
sq.add(Dense(8, activation='relu'))
sq.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
sq.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
sqq=sq.fit(x_train, y_train, epochs=150, batch_size=10
          ,validation_data=(x_test,y_test))

# evaluate the keras model
_, accuracy = sq.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))
predictions = sq.predict(x_test)


In [ ]:
plt.figure(figsize=[10,10])
plt.plot(sqq.history['loss'],'b',linewidth=3.0)
plt.plot(sqq.history['val_loss'],'r',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

# Exporting data

In [ ]:
#Export machine learning
pickle.dump(rf,open("drive/MyDrive/machileLearning.sav","wb"))

In [ ]:
#Export deep learning
with open("drive/MyDrive/deeplearning.json", "w") as json_file:
    model_json = kara.to_json()
    json_file.write(model_json)

In [ ]:
p = kara.predict(x_train)

In [ ]:
pickle.dump(scaler,open("drive/MyDrive/myscaler","wb"))

In [ ]:
y_train

In [ ]:
y_train[0]

In [ ]:
p.round()

In [ ]:
t=lr.predict([['a','a','a','a',
              'a','a','a','a']])
t[1]

In [ ]:
x_test[10]

In [ ]:
diabetes.iloc[766]

In [ ]:
x=x.drop('Outcome')

In [ ]:
x

In [ ]:
x=scaler.transform([x])

In [ ]:
rf.predict(x)

In [ ]:
np.argmax(sq.predict(x))
